Might be useful: 
- https://github.com/sdtaylor/pyPRISMClimate
- https://gist.github.com/dbr/3351090

https://prism.oregonstate.edu/documents/PRISM_formats.pdf

https://help.arcgis.com/en/arcgisdesktop/10.0/help/index.html#//009t00000010000000

In [7]:
import itertools
import os
import pickle
import tempfile
import zipfile

from pathlib import Path

import pandas as pd
import rasterio

In [6]:
!tree ./climate_data

./climate_data
├── max_temp
│   └── max_temp
│       ├── PRISM_tmax_stable_4kmD2_20040101_20041231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20050101_20051231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20060101_20061231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20070101_20071231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20080101_20081231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20090101_20091231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20100101_20101231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20110101_20111231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20120101_20121231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20130101_20131231_bil.zip
│       ├── PRISM_tmax_stable_4kmD2_20140101_20141231_bil.zip
│       └── PRISM_tmax_stable_4kmD2_20150101_20151231_bil.zip
├── max_vapor_pressure_deficit
│   └── max_vapor_pressure_deficit
│       ├── PRISM_vpdmax_stable_4kmD2_20040101_20041231_bil.zip
│       ├── PRISM_vpdmax_stable_4kmD2_20050101_20051231_bil.zip
│       ├── PRISM

In [39]:
extract_dir = Path('./extracted_data')
extract_dir.mkdir(parents=True, exist_ok=True)

This section of code tracks any files that have already been extracted, so that we don't extract them again.

In [8]:
existing_files = set()
for root, dirs, files in os.walk(str(extract_dir)):
    root_path = Path(root)
    for ff in files:
        existing_files.add(str(ff))

We recursively look into folders of zip files and only extract the `.hdr` and `.bil` files.
We also make sure that we don't extract anything that exists in the `extracted_dir` already.

In [9]:
for root, dirs, files in os.walk('climate_data'):
    root_path = Path(root)
    for ff in files:
        file_path = root_path / ff
        if file_path.suffix == '.zip':
            with zipfile.ZipFile(file_path, 'r') as archive:
                for afile in archive.filelist:
                    if afile.filename in existing_files:
                        continue
                    if afile.filename.endswith('.hdr') or afile.filename.endswith('.bil'):
                        src_file = archive.extract(afile, path=extract_dir)

We use `rasterio` to read a `.bil` file.

In [40]:
data_items = []
for root, dirs, files in os.walk(str(extract_dir)):
    root_path = Path(root)
    for ff in files:
        f_path = root_path / ff

        if f_path.suffix == '.bil':
            try:
                data = rasterio.open(f_path)
                data_items.append(data)
                print(data)
            except Exception as e:
                print(e)
            
            # this is only a test file and we stop after the first file we find
            break

<open DatasetReader name='extracted_data/PRISM_tmax_stable_4kmD2_20080921_bil.bil' mode='r'>


In [41]:
data_ary = data_items[0].read()
len(data_ary[data_ary!=-9999])

481631

In [42]:
print(data_items[0].height, data_items[0].width)

621 1405


In [49]:
data_items[0].bounds

BoundingBox(left=-125.02083333333336, bottom=24.062499999979053, right=-66.47916666661986, top=49.93749999999975)

In [50]:
data_items[0].bounds.left

-125.02083333333336

In [51]:
data_items[0].bounds.right

-66.47916666661986

In [52]:
data_items[0].bounds.top

49.93749999999975

In [53]:
data_items[0].bounds.bottom

24.062499999979053

In [54]:
delta_x = (data_items[0].bounds.right - data_items[0].bounds.left) / data_items[0].width
print(delta_x)

0.0416666666667


In [55]:
delta_y = (data_items[0].bounds.top - data_items[0].bounds.bottom) / data_items[0].height
print(delta_y)

0.0416666666667


We take a look at the contents of the header file.

In [15]:
!cat extracted_data/PRISM_tmax_stable_4kmD2_20080921_bil.hdr

BYTEORDER      I
LAYOUT         BIL
NROWS          621
NCOLS          1405
NBANDS         1
NBITS          32
BANDROWBYTES   5620
TOTALROWBYTES  5620
PIXELTYPE      FLOAT
ULXMAP         -125
ULYMAP         49.9166666666664
XDIM           0.0416666666667
YDIM           0.0416666666667
NODATA         -9999


Docs on the  rasterio [xy](https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html#rasterio.transform.TransformMethodsMixin.xy) function.

In [16]:
for data in data_items:
    data_ary = data.read()
    for ii in range(data.height):
        for jj in range(data.width):
            if data_ary[0,ii,jj] == -9999:
                # no data
                continue

            print(ii,jj,data.xy(ii,jj), data_ary[0,ii,jj])

            # I just want to look at the first non-empty entry
            raise Exception()

12 717 (-95.1249999999761, 49.416666666666) 13.322


Exception: 

In [125]:
(x0, y0) = data.xy(0,0)
(x1, y1) = data.xy(1,1)
(xn, yn) = data.xy(data.width, data.height)
print((x1-x0))
print((xn-x0))
print((xn-x0)/(x1-x0), data.height)
print((y1-y0))
print(yn-y0)
print((yn-y0)/(y1-y0), data.width)

0.041666666666699825
25.87500000002069
621.0000000000024 621
-0.041666666666699825
-58.541666666713496
1405.0000000000057 1405


In [95]:
data.xy(data.width, data.height)

(-99.12499999997931, -8.625000000047095)

In [94]:
data.bounds.top

49.93749999999975

In [115]:
data.bounds.bottom < data.bounds.top

True

In [119]:
data.read()[0,data.height-1, data.width-1]

-9999.0

In [123]:
(data.height, data.width)

(621, 1405)

In [122]:
data.read().shape

(1, 621, 1405)

In [120]:
?data.xy

Signature: data.xy(row, col, offset='center')
Docstring:
Returns the coordinates ``(x, y)`` of a pixel at `row` and `col`.
The pixel's center is returned by default, but a corner can be returned
by setting `offset` to one of `ul, ur, ll, lr`.

Parameters
----------
row : int
    Pixel row.
col : int
    Pixel column.
offset : str, optional
    Determines if the returned coordinates are for the center of the
    pixel or for a corner.

Returns
-------
tuple
    ``(x, y)``
File:      ~/.pyenv/versions/3.8.11/lib/python3.8/site-packages/rasterio/transform.py
Type:      method


In [66]:
(-95.1249999999761 - data.bounds.left - 0.5*delta_x) / delta_x

717.0000000000002

In [65]:
data.bounds.left + 717.5 * delta_x

-95.12499999997611

In [75]:
(49.416666666666 - data.bounds.top + 0.5*delta_y) / delta_y

-11.999999999999947

In [81]:
round((data.bounds.top - 0.5*delta_y - 49.416666666666) / delta_y)

12

In [91]:
(49.416666666666 - (data.bounds.bottom + 0.5*delta_y)) / delta_y

608.0

In [103]:
(49.416666666666 - y0) / (y1-y0)

12.0

In [23]:
min_x = float('+inf')
max_x = float('-inf')
min_y = float('+inf')
max_y = float('-inf')

for data in data_items:
    for ii in range(data.height+1):
        for jj in range(data.width+1):
            (xx,yy) = data.xy(ii,jj)
            min_x = min(min_x,xx)
            max_x = max(min_x,xx)
            min_y = min(min_y,yy)
            max_y = max(max_y,yy)

print(min_x, max_x)
print(min_y, max_y)

-125.0 -66.4583333332865
24.041666666645703 49.9166666666664
